In [24]:
# transformer plus cellular automata: conway game of life
import torch
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn 
from torch.nn import functional as F
from gameoflife import GameOfLife

batch_size = 8
block_size = 256 # Has to be the square of grid_size to read the full grid
max_iter = 2500
epochs = 10
eval_interval = 500
learning_rate = 5e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embed = 64
n_head = 8
n_layer = 16
dropout = 0
text = []

# create dictionaries and then define unique characters for encoding and decoding
tokens = ['0', '1'] #, 's', 'e']

# Example usage with cellular automata
grid_size = 16  # Grid size for the cellular automata
step_count = 1  # Number of steps to evolve the cellular automata

vocab_size=len(tokens)
stoi = { ch:i for i, ch in enumerate(tokens)}
itos = { i:ch for i, ch in enumerate(tokens)}
enc = lambda s: [stoi[c] for c in s]
dec = lambda l: ''.join([itos[i] for i in l])

def generate_random_input_string(size):
    """Generate a random grid as a string for a given grid size."""
    return ''.join(np.random.choice(tokens, size*size))

def generate_game_of_life_sequence(batch_size, grid_size, step_count):
    """Generate a batch of Game of Life initial states and final states."""
    initial_states = [generate_random_input_string(grid_size) for _ in range(batch_size)]
    final_states = []
    for state in initial_states:
        game = GameOfLife(input_string=state, generations=step_count)
        final_state = game.run_simulation()
        final_states.append(final_state)
    initial_states_with_tokens = [state for state in initial_states]
    return initial_states_with_tokens, final_states

# Define an appropriate size for your validation batch
val_batch_size = 20  

# load data
def get_batch(batch_size, grid_size, step_count, block_size):
    initial_states, final_states = generate_game_of_life_sequence(batch_size, grid_size, step_count)
    X = torch.tensor([enc(s)[:block_size] for s in initial_states], dtype=torch.long)
    Y = torch.tensor([enc(s)[:block_size] for s in final_states], dtype=torch.long)
    return X.to(device), Y.to(device)

# single head attention
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed,head_size,bias=False)
        self.query = nn.Linear(n_embed,head_size,bias=False)
        self.value = nn.Linear(n_embed,head_size,bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) *C**-0.5 # scaled attention
        # wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) # decoder block
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei@v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed,n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out =  torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.proj(out) # Projection si the linear transformation of the outcome of prev layer
        return out

class SinusoidalActivation(nn.Module):
    def forward(self, x):
        return torch.sin(x)
        # return x + torch.sin(x) ** 2

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed,4* n_embed), 
            # nn.ReLU(),
            nn.GELU(),
            # SinusoidalActivation(),
            nn.Linear(4* n_embed, n_embed),
            nn.Dropout(dropout),
            )
        self

    def forward(self,x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed //n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self,x):
        x = x + self.sa(self.ln1(x)) # add x for residual connections
        x = x + self.ffwd(self.ln1(x))
        return x

class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size,n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed,n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed,vocab_size)
        self.apply(self._init_weights)
        self.position_embedding_table_x = nn.Embedding(grid_size, n_embed)
        self.position_embedding_table_y = nn.Embedding(grid_size, n_embed)
        self.initial_conv = ConvLayer(1, n_embed, kernel_size=3, padding=1)  # Assuming n_embed is the desired output channel size

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def lossy(self, output, target):
        output = output.view(target.shape)  # Reshape output to match target's shape
        return torch.mean((output != target).float())

    def hamming_loss(self, predictions, targets):
        # Ensure predictions are binary (0 or 1)
        predictions = predictions > 0.5
        # Calculate mismatches
        mismatches = torch.ne(predictions.float(), targets.float()).float()
        # Calculate the average number of mismatches
        return torch.mean(mismatches)

    def forward(self,idx,targets=None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        # # In the forward method, apply this conv layer after embedding but before positional encoding
        # x = tok_emb.view(B, 1, grid_size, grid_size)  # Reshape for conv2d: [B, C, H, W]
        # x = self.initial_conv(x)
        # x = x.view(B, T, -1)  # Flatten back to the original shape for the transformer

        # Assuming your grid_size**2 = T
        grid_x, grid_y = torch.meshgrid(torch.arange(grid_size, device=device), torch.arange(grid_size, device=device), indexing='ij')
        pos_x = self.position_embedding_table_x(grid_x.flatten()).repeat(B, 1, 1)  # Repeat for batch size
        pos_y = self.position_embedding_table_y(grid_y.flatten()).repeat(B, 1, 1)  # Repeat for batch size
        pos_emb = pos_x + pos_y  # [B, T, C] matching tok_emb's dimensions
        x = tok_emb+pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        # print(f"logits are shape {logits.shape} are: {logits} for idx: {idx}")
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(-1, vocab_size)  # Reshape logits to [batch_size * block_size, vocab_size]
            targets = targets.view(-1,1)  # Flatten targets to [batch_size * block_size]
            targets = torch.cat((1 - targets, targets), dim=1)  
            # loss = self.lossy(logits, targets)
            # loss = F.cross_entropy(logits, targets)
            # predictions = torch.sigmoid(logits) > 0.5
            # loss = self.hamming_loss(predictions.view(-1), targets.view(-1))
            loss = F.binary_cross_entropy_with_logits(logits, targets.float(), reduction='mean')
            # loss = F.mse_loss(logits, F.one_hot(targets, num_classes=vocab_size).float())
            # print(f"logits are shape {logits.shape} are: {loss} for idx: {idx}")
        return logits, loss
    
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx, idx_next), dim = 1)
        return idx

class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)

    def forward(self, x):
        return F.relu(self.conv(x))

model = LanguageModel()
m = model.to(device)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.99, eps=1e-08, weight_decay=0.01, momentum=0.5, centered=False)
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate, rho=0.9, eps=1e-06, weight_decay=0.01)
# scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=1, verbose=True)
loss = None  # Initialize loss variable outside the loop

for epoch in range(epochs):
    for iter in range(max_iter // epochs):  # Distribute iterations across epochs
        model.train()
        xb, yb = get_batch(batch_size, grid_size, step_count, block_size)
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        max_norm = 0.9
        # Clip gradients to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()

        if iter % eval_interval == 0 and loss is not None:  # Validation logic
            model.eval()
            with torch.no_grad():
                xv, yv = get_batch(val_batch_size, grid_size, step_count, block_size)
                val_logits, val_loss = model(xv, yv)
                print(f"Epoch {epoch}, Iteration {iter}: Training Loss = {loss.item()}, Validation Loss = {val_loss.item()}")
            model.train()

    scheduler.step(val_loss)  # Update the learning rate at the end of each epoch

# Save:
torch.save(model, 'cat_cgol_model_b.pth')


Epoch 0, Iteration 0: Training Loss = 0.6942821145057678, Validation Loss = 0.6954941749572754
Epoch 1, Iteration 0: Training Loss = 0.6993380784988403, Validation Loss = 0.6965514421463013
Epoch 2, Iteration 0: Training Loss = 0.6947116851806641, Validation Loss = 0.6969835162162781
Epoch 00003: reducing learning rate of group 0 to 1.0000e-04.
Epoch 3, Iteration 0: Training Loss = 0.6968087553977966, Validation Loss = 0.6965460777282715
Epoch 4, Iteration 0: Training Loss = 0.6956965923309326, Validation Loss = 0.697391152381897
Epoch 00005: reducing learning rate of group 0 to 2.0000e-05.
Epoch 5, Iteration 0: Training Loss = 0.6944871544837952, Validation Loss = 0.6969743967056274
Epoch 6, Iteration 0: Training Loss = 0.6947259902954102, Validation Loss = 0.6957005262374878
Epoch 00007: reducing learning rate of group 0 to 4.0000e-06.
Epoch 7, Iteration 0: Training Loss = 0.695305347442627, Validation Loss = 0.6963487267494202
Epoch 8, Iteration 0: Training Loss = 0.6966075897216797

In [26]:
import numpy as np
import matplotlib.pyplot as plt

model = torch.load('cat_cgol_model_b.pth')
model.eval()
input_length = 8
generations = 1
np.random.seed(42)  # Seed for reproducibility
input_sequence = ''.join(np.random.choice(tokens, input_length*input_length))
print(f"Input is: {input_sequence}")
context = torch.tensor(enc(input_sequence), dtype=torch.long, device=device).unsqueeze(0)
output = model.generate(context, max_new_tokens=len(input_sequence))
generated_text_t = dec(output[0].tolist())
game = GameOfLife(input_sequence, generations=generations)

generated_text_ca = game.run_simulation()

def visualize_grid_with_modifiers(grid):
    """Visualise the grid."""
    base_colors = {'0': 'red', '1': 'blue', 's': 'grey', 'e': 'black'}
    colors = []
    for row in grid:
        row_colors = [base_colors[base] for base in row]
        colors.extend(row_colors)

    plt.figure(figsize=(20, 2))
    plt.bar(range(len(colors)), np.ones(len(colors)), color=colors)
    plt.axis('off')
    plt.title('Visualization')
    plt.show()

print(f"Generated from CA is: {generated_text_ca}\n")
print(f"Generated from T is: {(generated_text_t[len(input_sequence):])}\n")
visualize_grid_with_modifiers(input_sequence)
visualize_grid_with_modifiers(generated_text_ca)
visualize_grid_with_modifiers(generated_text_t[len(input_sequence):])  # Use the fixed, split decoded output

print(len(generated_text_t))
print(len(input_sequence))
print(len(generated_text_t)-(len(input_sequence)))

Input is: 0100010001000010111010111111110011101000001111101101010110000000


RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 1